### Pre-Processing - Loan Offer Predictive Model
This notebook is dedicated to loading in our data, performing a few cleaning steps, and ensuring that it's split appropriately for modeling.

In [1]:
import sqlite3
import pandas as pd
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
with sqlite3.connect('bank.sqlite') as conn:
    df = pd.read_sql_query('SELECT * FROM clean_table', conn)

df.head()

,age,experience,income,zip,family,cc_avg,education,mortgage,personal_loan,securiities_account,cd_account,online,credit_card
0,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [3]:
# Saw in EDA that ZIP Code didn't have a huge impact on loan offers being accepted. Removing that.
df = df.drop('zip', axis=1)

In [4]:
# Heavily imbalanced dataset in terms of what we're classifying
df.personal_loan.value_counts()

0    4468
1     480
Name: personal_loan, dtype: int64

In [5]:
# Split data into X and y
X = df.drop('personal_loan', axis =1 )
y = df.personal_loan

In [6]:
# Scale X
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [7]:
# Split X and y
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = .25, random_state = 5)

In [8]:
# Use SMOTE to address imalanced data
oversample = SMOTE(random_state = 5)
X_train_res, y_train_res = oversample.fit_sample(X_train, y_train)

In [9]:
# We see before SMOTE that rejected offers represented roughly 9x as many observations in training as accepted offers
print('Accepted loan offers in training before SMOTE: {}'.format(sum(y_train == 1)))
print('Rejected loan offers in training before SMOTE: {}'.format(sum(y_train == 0)))

Accepted loan offers in training before SMOTE: 373
Rejected loan offers in training before SMOTE: 3338


In [10]:
# With SMOTE, we synthetically oversample to get accepted offers to be the same as rejected offers
print('Accepted loan offers in training after SMOTE: {}'.format(sum(y_train_res == 1)))
print('Rejected loan offers in training after SMOTE: {}'.format(sum(y_train_res == 0)))

Accepted loan offers in training after SMOTE: 3338
Rejected loan offers in training after SMOTE: 3338


In [11]:
# We see a corresponding increase in rows for X_train_res now that we've used SMOTE
print(X_train.shape)
print(X_train_res.shape)

(3711, 11)
(6676, 11)
